# Tutorial 4: Simplification Rules

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/buildLittleWorlds/types-pure-passage-calculus/blob/main/notebooks/tutorial_04_simplification_rules.ipynb)

---

## When Is a Passage Fully Simplified?

In Year 731, Kelleth Mund's treatise *The Passage Calculus* introduced the concept of **normal form**:

> *"A passage is in normal form when no further simplification is possible. It has reached its essential structure."*

This tutorial explores the rules that govern simplification and when passages are considered "done."

---

## Learning Objectives

By the end of this tutorial, you will:
1. Identify passages in **normal form**
2. Distinguish **weak head normal form** from **normal form**
3. Understand **eta reduction**
4. Analyze reduction behavior using the dataset

## Setup

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

BASE_URL = "https://raw.githubusercontent.com/buildLittleWorlds/densworld-datasets/main/data/"

reductions = pd.read_csv(BASE_URL + "passage_reductions.csv")

print(f"Loaded {len(reductions)} reduction steps")

## Part 1: What Is Normal Form?

A passage is in **normal form** (NF) when it contains no redexes—no subexpressions of the form `(λx.body) arg`.

### Examples

**In normal form:**
- `x` (just a variable)
- `λx.x` (abstraction, no redex inside)
- `λx.λy.x` (nested abstractions, no redexes)
- `x y` (application, but neither part is a λ)

**NOT in normal form:**
- `(λx.x) y` (this is a redex!)
- `λz.((λx.x) z)` (contains a redex inside)

In [ ]:
# Find all reduction steps that reach normal form
normal_forms = reductions[reductions['is_normal_form'] == True]
print(f"Steps reaching normal form: {len(normal_forms)}")
normal_forms[['reduction_id', 'expression_after', 'total_steps', 'notes']].head(10)

In [ ]:
# What kinds of expressions end up in normal form?
# Let's categorize by what the final expression looks like

def categorize_normal_form(expr):
    expr = str(expr).strip()
    if expr.startswith('λ') or expr.startswith('(λ'):
        return 'abstraction'
    elif len(expr) == 1 and expr.isalpha():
        return 'variable'
    elif '(' in expr:
        return 'application'
    else:
        return 'other'

normal_forms['nf_type'] = normal_forms['expression_after'].apply(categorize_normal_form)
normal_forms['nf_type'].value_counts()

## Part 2: Weak Head Normal Form (WHNF)

Sometimes we don't need to fully reduce an expression. **Weak Head Normal Form** is reached when:
- The expression is a λ-abstraction, OR
- The expression is an application where the function is not a λ

WHNF stops as soon as we can't reduce the "head" (outermost) position.

### Example

`λz.((λx.x) z)` is in WHNF (it's a λ) but NOT in normal form (the body contains a redex).

In [ ]:
# Many functional languages (Haskell) use WHNF for lazy evaluation
# Let's look at cases where the Theta combinator reaches WHNF

theta_traces = reductions[reductions['notes'].str.contains('Theta|weak head', case=False, na=False)]
theta_traces[['reduction_id', 'expression_after', 'is_normal_form', 'notes']]

## Part 3: Eta Reduction

Besides beta reduction, there's another simplification: **eta reduction**.

The eta rule:
```
λx.(f x)  →  f    (when x is not free in f)
```

This says: if a function just applies `f` to its argument, it's the same as `f`.

### Example
- `λx.(add x)` eta-reduces to `add`
- `λx.((λy.y) x)` can first beta-reduce the inner part, then eta-reduce

In [ ]:
# Find eta reductions in the data
eta_steps = reductions[reductions['rule_applied'] == 'eta']
print(f"Eta reduction steps: {len(eta_steps)}")
eta_steps[['reduction_id', 'expression_after', 'notes']]

In [ ]:
# Eta equivalence in Python
def add(x):
    return x + 1

# These are eta-equivalent:
f1 = add
f2 = lambda x: add(x)

print(f"add(5) = {add(5)}")
print(f"(λx.add x)(5) = {f2(5)}")
print(f"Same result: {add(5) == f2(5)}")

## Part 4: Non-Termination

Not all expressions have a normal form. Some reduce forever.

The classic example is **Omega**:
```
Ω = (λx.x x)(λx.x x)
  → (λx.x x)(λx.x x)
  → (λx.x x)(λx.x x)
  → ...
```

In [ ]:
# Find non-terminating reductions
non_terminating = reductions[reductions['terminates'] == False].drop_duplicates('reduction_id')
print(f"Non-terminating expressions: {len(non_terminating)}")
non_terminating[['reduction_id', 'initial_expression', 'notes']].head(10)

In [ ]:
# The Omega reduction
omega_trace = reductions[reductions['reduction_id'] == 'PR-009']
omega_trace[['step_number', 'expression_after', 'terminates', 'notes']]

### Why Non-Termination Matters

Non-termination is related to the Great Categorical Collapse. When documents could reference themselves in certain ways, the Archive's indexing system would loop forever—just like Omega.

In [ ]:
# But not all self-reference leads to non-termination!
# The Y combinator enables useful recursion

# Find Y combinator traces
y_traces = reductions[reductions['notes'].str.contains('Y comb', case=False, na=False)]
y_traces[['reduction_id', 'initial_expression', 'terminates', 'notes']].drop_duplicates('reduction_id')

## Part 5: Reduction Statistics

In [ ]:
# How many reductions terminate vs don't?
final_steps = reductions[reductions['is_normal_form'] == True]
termination_counts = final_steps['terminates'].value_counts()

plt.figure(figsize=(8, 5))
termination_counts.plot(kind='bar', color=['steelblue', 'coral'])
plt.title('Terminating vs Non-Terminating Expressions')
plt.xlabel('Terminates')
plt.ylabel('Count')
plt.xticks(rotation=0)
plt.show()

In [ ]:
# What rules are applied most often?
reductions['rule_applied'].value_counts()

In [ ]:
# Average steps by termination status
terminating = final_steps[final_steps['terminates'] == True]

print(f"Average steps for terminating expressions: {terminating['total_steps'].mean():.1f}")
print(f"Max steps for terminating expressions: {terminating['total_steps'].max()}")

In [ ]:
# What's the longest terminating reduction?
longest = terminating[terminating['total_steps'] == terminating['total_steps'].max()]
longest[['reduction_id', 'initial_expression', 'total_steps', 'notes']]

## Part 6: Normal vs Applicative Order

In [ ]:
# Some expressions terminate in normal order but not applicative order

# K = λx.λy.x (returns first argument, ignores second)
# Ω = (λx.x x)(λx.x x) (never terminates)

# Normal order: (K a) Ω → a  (never evaluates Ω)
# Applicative order: (K a) Ω → stuck trying to evaluate Ω

strategy_examples = reductions[reductions['notes'].str.contains('Normal order|Applicative', case=False, na=False)]
strategy_examples[['reduction_id', 'initial_expression', 'terminates', 'notes']].drop_duplicates('reduction_id')

In [ ]:
# Python uses applicative order (eager evaluation)
# This is why we can't directly implement lazy lambda calculus

def K(x):
    return lambda y: x

def omega():
    while True:
        pass  # Infinite loop

# This would hang in Python:
# K(5)(omega())  # Python tries to evaluate omega() first!

# But this works:
print(K(5)(lambda: None))  # Passing a function, not calling it

## Exercises

### Exercise 1: Identify Normal Forms

Which of these are in normal form?

1. `λx.x`
2. `(λx.x) y`
3. `x y`
4. `λf.λx.f (f x)`
5. `λx.((λy.y) x)`

In [ ]:
# Exercise 1 workspace

# 1. λx.x - In NF? 
#    Is there a redex (λ..)arg anywhere?

# 2. (λx.x) y - In NF?
#    This IS a redex...

# 3. x y - In NF?
#    Neither x nor y is a λ...

# 4. λf.λx.f (f x) - In NF?
#    Look inside the body...

# 5. λx.((λy.y) x) - In NF?
#    The body contains...

### Exercise 2: Eta Reduction

Which can be eta-reduced? If so, to what?

1. `λx.(f x)`
2. `λx.(x x)`
3. `λx.((g y) x)`
4. `λx.(f x y)`

In [ ]:
# Exercise 2 workspace

# Eta rule: λx.(f x) → f when x not free in f

# 1. λx.(f x)
#    Is x free in f? No (f is just a variable)
#    → f

# 2. λx.(x x)
#    This is NOT of the form λx.(f x) where f doesn't contain x
#    The first x IS x, which we're abstracting over

# 3. λx.((g y) x)
#    Is this λx.(f x) where f = (g y)?

# 4. λx.(f x y)
#    This is λx.((f x) y), not λx.(f x)

### Exercise 3: Analyze Reduction Patterns

Using the reductions dataset:
1. Find all unique expressions that reduce to just a variable
2. Find the expression with the most reduction steps
3. Calculate the percentage of expressions that terminate

In [ ]:
# Exercise 3 workspace

# 1. Expressions reducing to a variable
# Hint: filter is_normal_form==True, then check if expression_after is a single letter

# 2. Most reduction steps
# Hint: find max of total_steps for terminating expressions

# 3. Termination percentage
# Hint: count terminates==True vs total unique reduction_ids

## Summary

In this tutorial, we learned:

1. **Normal form** means no redexes remain—fully simplified
2. **Weak head normal form (WHNF)** stops when the head can't reduce
3. **Eta reduction** `λx.(f x) → f` removes pointless wrapping
4. **Non-termination** occurs when reductions loop forever (like Omega)
5. **Normal order** can find normal forms that applicative order misses

### Next Tutorial

In Tutorial 5, we'll discover that **everything can be encoded as passages**—numbers, booleans, pairs, and lists. This is Mund's most remarkable insight: objects are unnecessary.